## Proof of Concept
The output is the df that gets generated, to be then made into a csv. I've set it up to 10... but it can be set to something larger. 

In [2]:
import os
import requests
import json
import pandas as pd
import random

def calculate_price(drink):
    premium_ingredients = ["Absinthe", "Grand Marnier", "Cointreau", "Chambord", "Campari",
                            "Green Chartreuse", "Añejo Rum", "Midori Melon Liqueur", "Frangelico",
                            "Galliano", "St. Germain Elderflower Liqueur", "Bailey's Irish Cream",
                            "Kahlua", "Amaretto", "Drambuie", "Benedictine", "Dom Pérignon",
                            "Patrón Tequila", "Château Margaux", "Hennessy Cognac"]

    base_price = 6.5 

    # Additional pricing based on factors
    price_per_ingredient = 0.75
    premium_charge = 2.5
    alcoholic_price_increase = 3 

    price = base_price + drink['num_ingredients'] * price_per_ingredient

    # surcharge for specific spirits
    for ingredient in drink['ingredients'].split(', '):
        if ingredient in premium_ingredients:
            price += premium_charge
            # Add a random factor for each premium ingredient
            price += random.uniform(-0.25, 1)

    # Add additional price for alcoholic drinks
    if drink['alcoholic']:
        price += alcoholic_price_increase

    # Add a general random factor
    price += random.uniform(-1, 1)

    return round(price, 2)


def generate_cocktail_dataframe(num_cocktails):
    cocktail_id = [] 
    names = []
    ingredients = [] 
    num_ingredients = []  
    alcoholic = []
    acidic = []
    caffiene = [] 
    category = []  
    glass = []  
    sugary = []
    instructions = []  
    thumb_urls = []  

    sweet_list = [
    'Grenadine', 'Pineapple juice', 'Amaretto', 'Sour mix',
    'Kahlua', 'Powdered sugar', 'Banana Liqueur', 'Chocolate liqueur', 
    'Peach schnapps', 'Orange Juice', 'Soda water', 'Vanilla syrup',
    'Baileys irish cream', 'White Creme de Menthe', 'Sprite'
    'Creme de Cacao','Surge', 'Honey', 'Cocoa powder', ' 7-up',
    'Brandy', 'Cranberry juice', 'Ginger ale', 'Peach Vodka', 'Coca-Cola', 'Coke',
    'Creme de Cassis', 'Champagne', 'Angostura bitters', 'Tonic water',
    'Grapefruit Juice', 'Agave Syrup', 'Soda Water', 'Zima',
    'Chambord raspberry liqueur', 'Vodka', 'Sugar', 'Sugar Syrup',
    'Passion fruit juice', 'Prosecco', 'Elderflower cordial', 'Jägermeister',
    'Root beer', 'Corn syrup', 'syrup', 'Limeade', 
    'Butterscotch schnapps', 'Vanilla Ice-Cream',
    'Sherry', 'Grand Marnier', 'Orange juice', 'Lemonade', 'Orange spiral',
    'Sweet Vermouth', 'Sloe gin', 'Apricot brandy', 'Raspberry syrup', 'Dry Vermouth'
    ]


    url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

    # Generate random cocktails that are valid
    while len(cocktail_id) < num_cocktails:
        r = requests.get(url)

        if r.status_code == 200:
            site_json = r.json()

            # Check if the response contains valid data
            if 'drinks' in site_json and site_json['drinks']:
                drink = site_json['drinks'][0]
                name = drink.get('strDrink')

                # Append the columns 
                cocktail_id.append(drink['idDrink'])
                names.append(name)

                # Extract ingredients (removing Nones)
                ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]

                ingredients_lower = [ingredient.lower() for ingredient in ingredients_raw if ingredient]
                
                ingredients.append(', '.join(filter(None, ingredients_raw)))
                
                

                # Calculate and append the number of ingredients
                num_ingredients.append(len(list(filter(None, ingredients_raw))))

                # Convert "strAlcoholic" to boolean
                alcoholic_status = drink.get('strAlcoholic', '').lower() == 'alcoholic'
                alcoholic.append(alcoholic_status)

                # Check if any of the specified ingredients are in the list
                acidic_ingredients = ["lemon", "lime", "orange", "lemon juice", "lime juice", "orange juice", "peel"]

                acidic_status = any(ingredient in acidic_ingredients for ingredient in ingredients_lower)
                acidic.append(acidic_status)

                caf_status = any(ingredient in ["Coffee", "Espresso"] for ingredient in ingredients_raw) 
                caffiene.append(caf_status)

                category.append(drink.get('strCategory', ''))
                glass.append(drink.get('strGlass', ''))

                sugar_score = 0
                sugar_score = sum(1 for ingredient in ingredients_raw if ingredient in sweet_list)
                sugary.append(sugar_score)

                instructions.append(drink.get('strInstructions', ''))
                thumb_urls.append(drink.get('strDrinkThumb', ''))


            else:
                continue

    # Create DataFrame
    df_cocktail = pd.DataFrame({
        "cocktail_id": cocktail_id,
        "cocktail_name": names,
        "num_ingredients": num_ingredients,
        "ingredients": ingredients,
        "alcoholic": alcoholic,
        "sugary_score":sugary,
        "acidic": acidic,
        "caffiene": caffiene, 
        "category": category, 
        "glass": glass,
        "instructions":instructions,
        "image_url": thumb_urls
    })  

    df_cocktail["price"] =  df_cocktail.apply(calculate_price, axis=1)

    # Drop duplicates based on 'cocktail_id'
    df_cocktail = df_cocktail.drop_duplicates(subset=['cocktail_id'])   

    csv_filename = "cocktails_result.csv"
    csv_path = os.path.join(os.getcwd(), csv_filename)
    df_cocktail.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")

    pd.set_option('display.max_colwidth', None) 

    return df_cocktail


generate_cocktail_dataframe(10)



Data saved to c:\Users\andre\Desktop\Personal\CocktailDB_API\cocktails_result.csv


,cocktail_id,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,category,glass,instructions,image_url,price
0,11338,English Highball,5,"Brandy, Gin, Sweet Vermouth, Carbonated water, Lemon peel",True,2,False,False,Ordinary Drink,Highball glass,"Pour brandy, gin, and sweet vermouth into a highball glass over ice cubes. Fill with carbonated water. Add the twist of lemon peel, stir, and serve. (Ginger ale may be substituted for carbonated water, if preferred.)",https://www.thecocktaildb.com/images/media/drink/dhvr7d1504519752.jpg,12.78
1,11410,Gin Fizz,4,"Gin, Lemon, Powdered sugar, Carbonated water",True,1,True,False,Ordinary Drink,Highball glass,"Shake all ingredients with ice cubes, except soda water. Pour into glass. Top with soda water.",https://www.thecocktaildb.com/images/media/drink/drtihp1606768397.jpg,13.27
2,12802,Cranberry Cordial,3,"Cranberries, Sugar, Light rum",True,1,False,False,Homemade Liqueur,Collins Glass,"Place the chopped cranberries in a 2 liter jar that has a tight-fitting lid. Add the sugar and rum. Adjust the lid securely and place the jar in a cool, dark place. Invert the jar and shake it every day for six weeks. Strain the cordial into bottles and seal with corks.",https://www.thecocktaildb.com/images/media/drink/qtspsx1472667392.jpg,11.13
3,11014,Alexander,4,"Gin, Creme de Cacao, Light cream, Nutmeg",True,0,False,False,Ordinary Drink,Cocktail glass,Shake all ingredients with ice and strain contents into a cocktail glass. Sprinkle nutmeg on top and serve.,https://www.thecocktaildb.com/images/media/drink/0clus51606772388.jpg,13.06
4,11424,Godmother,2,"Vodka, Amaretto",True,2,False,False,Ordinary Drink,Old-fashioned glass,Pour vodka and amaretto into an old-fashioned glass over ice and serve.,https://www.thecocktaildb.com/images/media/drink/quksqg1582582597.jpg,15.27
5,13072,Popped cherry,4,"Vodka, Cherry liqueur, Cranberry juice, Orange juice",True,3,True,False,Ordinary Drink,Highball glass,Served over ice in a tall glass with a popped cherry (can add more popped cherries if in the mood)!,https://www.thecocktaildb.com/images/media/drink/sxvrwv1473344825.jpg,12.88
6,14782,Hot Creamy Bush,3,"Irish whiskey, Baileys irish cream, Coffee",True,1,False,True,Coffee / Tea,Irish coffee cup,Combine all ingredients in glass,https://www.thecocktaildb.com/images/media/drink/spvrtp1472668037.jpg,12.51
7,12656,Banana Strawberry Shake,5,"Strawberries, Banana, Yoghurt, Milk, Honey",False,1,False,False,Shake,Highball Glass,Blend all together in a blender until smooth.,https://www.thecocktaildb.com/images/media/drink/vqquwx1472720634.jpg,10.88
8,13328,Chocolate Milk,3,"Chocolate liqueur, Milk, Amaretto",True,2,False,False,Shot,Shot Glass,"Put the milk in the bottom, pour the Liquer on top and add the dash of amaretto. Do not mix. SLAM IT!",https://www.thecocktaildb.com/images/media/drink/j6q35t1504889399.jpg,15.39
9,17105,501 Blue,5,"Blue Curacao, Blueberry schnapps, Vodka, Sour mix, 7-Up",True,2,False,False,Ordinary Drink,Collins Glass,Mix equal amounts into a glass with ice.,https://www.thecocktaildb.com/images/media/drink/ywxwqs1461867097.jpg,13.52


# Grabbing a Bunch of Beverages (500)

In [7]:
df_cocktails = generate_cocktail_dataframe(500)
df_cocktails

Data saved to c:\Users\andre\Desktop\Personal\CocktailDB_API\cocktails_result.csv


,cocktail_id,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,category,glass,instructions,image_url,price
0,14360,Absolut Sex,4,"Absolut Kurant, Midori melon liqueur, Cranberry juice, Sprite",True,1,False,False,Shot,Old-fashioned glass,"Shake Absolut Kurant, Midori, and Cranberry juice in shaker with ice: Strain into rocks glass: Splash of seven on top.Absolut Sex.",https://www.thecocktaildb.com/images/media/drink/xtrvtx1472668436.jpg,13.50
1,14538,Bumble Bee,3,"Baileys irish cream, Kahlua, Sambuca",True,2,False,False,Shot,Shot glass,"This is a layered shot. First pour the Bailey's into the shot glass. Then take an upside down spoon and touch it to the inside wall of the glass. Carefully add the Kahlua. Repeat this process for the Sambuca. If done properly, the alcohol will stay separated and resemble a bumble bee. Enjoy!!!",https://www.thecocktaildb.com/images/media/drink/uwqpvv1461866378.jpg,15.70
2,178359,Kiwi Martini,4,"Kiwi, Sugar Syrup, Vodka, Kiwi",True,2,False,False,Cocktail,Cocktail glass,"The kiwi martini is a very fun vodka cocktail and it is one of the best drinks that makes use of fresh fruit. Though there are a few recipes floating around, this is one of the easiest and it is an absolutely delightful green martini to drink.\r\n\r\nFor this recipe, you'll simply muddle slices of kiwi with simple syrup, then shake it with vodka. It's a drink that anyone can mix up in minutes and a perfect cocktail to show off your favorite vodka.",https://www.thecocktaildb.com/images/media/drink/bmxmyq1630407098.jpg,13.30
3,178354,Pink Moon,5,"Gin, Coconut Liqueur, Elderflower cordial, Lime Juice, Blackberries",True,1,True,False,Cocktail,Whiskey Glass,"Slowly shake in a shaker with ice, strain into a square whiskey glass. Top with fresh ice. Add the blackberries to garnish. Add flowers and a green leaf for a special look!",https://www.thecocktaildb.com/images/media/drink/lnjoc81619696191.jpg,13.79
4,14978,Rum Punch,5,"Rum, Ginger ale, Fruit punch, Orange juice, Ice",True,2,True,False,Punch / Party Drink,Punch bowl,Mix all ingredients in a punch bowl and serve.,https://www.thecocktaildb.com/images/media/drink/wyrsxu1441554538.jpg,12.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,15997,GG,3,"Galliano, Ginger ale, Ice",False,1,False,False,Ordinary Drink,Collins Glass,Pour the Galliano liqueur over ice. Fill the remainder of the glass with ginger ale and thats all there is to it. You now have a your very own GG.,https://www.thecocktaildb.com/images/media/drink/vyxwut1468875960.jpg,12.24
494,11005,Dry Martini,3,"Gin, Dry Vermouth, Olive",True,1,False,False,Cocktail,Cocktail glass,"Straight: Pour all ingredients into mixing glass with ice cubes. Stir well. Strain in chilled martini cocktail glass. Squeeze oil from lemon peel onto the drink, or garnish with olive.",https://www.thecocktaildb.com/images/media/drink/6ck9yi1589574317.jpg,11.18
496,17268,Blue Hurricane,6,"Rum, Dark Rum, Passoa, Blue Curacao, Sweet and Sour, Ice",True,0,False,False,Cocktail,Highball glass,If each part is 1/2 oz then use about 2.5 cups of ice.\r\n\r\nBlend it all together. \r\nDrink it with a big straw if you have one.\r\n\r\n,https://www.thecocktaildb.com/images/media/drink/nwx02s1515795822.jpg,13.05
497,17267,Bahama Mama,7,"Rum, Dark Rum, Banana liqueur, Grenadine, Pineapple Juice, Orange Juice, Sweet and Sour",True,2,True,False,Cocktail,Highball glass,Add 2 parts club soda or more or less to taste.\r\n\r\nMix it all together and pour over a bunch of ice. Drink with a straw.,https://www.thecocktaildb.com/images/media/drink/tyb4a41515793339.jpg,14.79


In [4]:
# Recommendation System 
def drink_rec(num_ingred: tuple, alc, spec_ingred, sugar, acid, caf, price=20):


    df_cocktails = pd.read_csv('cocktails_result_349.csv')

    # Making ingredient selection case insensitive! 
    df_cocktails['ingredients_lower'] = df_cocktails['ingredients'].str.lower()
    spec_ingred_lower = spec_ingred.lower()

    df_rec = df_cocktails[
        (df_cocktails['num_ingredients'].between(num_ingred[0], num_ingred[1])) &
        (df_cocktails['alcoholic'] == alc) & 
        (df_cocktails['sugary_score'] == sugar) & 
        (df_cocktails['acidic'] == acid) &
        (df_cocktails['caffiene'] == caf) & 
        (df_cocktails['price'] < price) &
        (df_cocktails['ingredients_lower'].str.contains(spec_ingred_lower))
    ] 

    df_rec = df_rec.drop(columns=['ingredients_lower'])  # Assign the result back to df_rec

    # TEMPORARILY setting the column width size to None... will revert after function call
    pd.set_option('display.max_colwidth', None)

    # return df_rec[['cocktail_name', 'num_ingredients', 'ingredients', 'alcoholic', 'sugary_score', 'acidic', 'caffiene', 'glass', 'instructions', 'image_url', 'price']]
    return df_rec[['cocktail_name', 'num_ingredients', 'ingredients', 'alcoholic', 'sugary_score', 'acidic', 'caffiene', 'glass', 'instructions', 'price']]

pd.reset_option("display.max_colwidth")

drink_rec((2,10), True, "whiskey", 1, False, False, 20)

,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,glass,instructions,price
209,Jack's Vanilla Coke,4,"Ice, Tennessee whiskey, Vanilla extract, Coca-Cola",True,1,False,False,Old-fashioned glass,"After pouring in your ingredients, and adding 3-5 ice cubes, according to taste. Stir the drink with a stirrer to get the Vanilla off the bottom.",11.93
215,Boulevardier,4,"Campari, Sweet Vermouth, Rye whiskey, Orange Peel",True,1,False,False,Martini Glass,"Stir with ice, strain, garnish and serve.",15.93
218,Owen's Grandmother's Revenge,4,"Whiskey, Beer, Lemonade, Ice",True,1,False,False,Highball glass,Add ingredients and mix in blender.,13.08
241,The Jimmy Conway,3,"Irish Whiskey, Amaretto, Cranberry Juice",True,1,False,False,Whiskey sour glass,Fill glass with ice\r\nPour in The Irishman and Disaronno\r\nFill to the top with Cranberry Juice\r\nGarnish with a slice of lemon…Enjoy!,13.98


# Final Product 

In [6]:
import tkinter as tk
from tkinter import ttk
from IPython.display import display
import pandas as pd

def drink_rec(min_ingred:int, max_ingred:int, alc:bool, spec_ingred:str, sugar:int, acid:bool, caf:bool, price:float):
    df_cocktails = pd.read_csv('cocktails_result_349.csv')

    print("Input Parameters:")
    print("Min Number of Ingredients:", min_ingred)
    print("Max Number of Ingredients:", max_ingred)
    print("Alcoholic:", alc)
    print("Special Ingredient:", spec_ingred)
    print("Sugary Score:", sugar)
    print("Acidic:", acid)
    print("Caffeine:", caf)
    print("Price:", price)

    # Making ingredient selection case insensitive! 
    df_cocktails['ingredients_lower'] = df_cocktails['ingredients'].str.lower()
    spec_ingred_lower = spec_ingred.lower()

    df_rec = df_cocktails[
        (df_cocktails['num_ingredients'].between(min_ingred, max_ingred)) &
        (df_cocktails['alcoholic'] == alc) & 
        (df_cocktails['sugary_score'] == sugar) & 
        (df_cocktails['acidic'] == acid) &
        (df_cocktails['caffiene'] == caf) & 
        (df_cocktails['price'] < price) &
        (df_cocktails['ingredients_lower'].str.contains(spec_ingred_lower))
    ] 

    df_rec = df_rec.drop(columns=['ingredients_lower'])  # Assign the result back to df_rec

    return df_rec[['cocktail_name', 'num_ingredients', 'ingredients', 'alcoholic', 'sugary_score', 'acidic', 'caffiene', 'glass', 'instructions', 'price']]

def submit():
    min_ingred = min_num_ingred_entry.get()
    max_ingred = max_num_ingred_entry.get()
    alc_val = alc_var.get()
    spec_ingred_val = spec_ingred_entry.get()
    sugar_val = sugar_var.get()
    acid_val = acid_var.get()
    caf_val = caf_var.get()
    price_val = price_entry.get()

    # Call drink_rec function with user inputs
    recommendations = drink_rec(int(min_ingred), int(max_ingred), alc_val == "True", spec_ingred_val, int(sugar_val), acid_val == "True", caf_val == "True", float(price_val))
    root.destroy()  # Close the UI window

    display(recommendations)

    return recommendations

    
def get_radio_selection(variable):
    return True if variable.get() == "True" else False

root = tk.Tk()
root.title("Cocktail Preferences")

# Min number of ingredients
min_num_ingred_label = ttk.Label(root, text="Min Number of ingredients:")
min_num_ingred_label.grid(row=0, column=0, padx=5, pady=5)
min_num_ingred_entry = ttk.Entry(root)
min_num_ingred_entry.grid(row=0, column=1, padx=5, pady=5)

# Max number of ingredients
max_num_ingred_label = ttk.Label(root, text="Max Number of ingredients:")
max_num_ingred_label.grid(row=1, column=0, padx=5, pady=5)
max_num_ingred_entry = ttk.Entry(root)
max_num_ingred_entry.grid(row=1, column=1, padx=5, pady=5)

# Alcoholic
alc_label = ttk.Label(root, text="Alcoholic:")
alc_label.grid(row=2, column=0, padx=5, pady=5)
alc_var = tk.StringVar(value="True")
alc_true_radio = ttk.Radiobutton(root, text="True", variable=alc_var, value="True")
alc_false_radio = ttk.Radiobutton(root, text="False", variable=alc_var, value="False")
alc_true_radio.grid(row=2, column=1, padx=5, pady=5)
alc_false_radio.grid(row=2, column=2, padx=5, pady=5)

# Special ingredient
spec_ingred_label = ttk.Label(root, text="Special ingredient:")
spec_ingred_label.grid(row=3, column=0, padx=5, pady=5)
spec_ingred_entry = ttk.Entry(root)
spec_ingred_entry.grid(row=3, column=1, padx=5, pady=5)

# Sugary level
sugar_label = ttk.Label(root, text="Sugary level:")
sugar_label.grid(row=4, column=0, padx=5, pady=5)
sugar_var = tk.StringVar(value="0")
sugar_dropdown = ttk.Combobox(root, textvariable=sugar_var, values=["0", "1", "2", "3", "4"])
sugar_dropdown.grid(row=4, column=1, padx=5, pady=5)

# Acidic
acid_label = ttk.Label(root, text="Acidic:")
acid_label.grid(row=5, column=0, padx=5, pady=5)
acid_var = tk.StringVar(value="True")
acid_true_radio = ttk.Radiobutton(root, text="True", variable=acid_var, value="True")
acid_false_radio = ttk.Radiobutton(root, text="False", variable=acid_var, value="False")
acid_true_radio.grid(row=5, column=1, padx=5, pady=5)
acid_false_radio.grid(row=5, column=2, padx=5, pady=5)

# Caffeinated
caf_label = ttk.Label(root, text="Caffeinated:")
caf_label.grid(row=6, column=0, padx=5, pady=5)
caf_var = tk.StringVar(value="True")
caf_true_radio = ttk.Radiobutton(root, text="True", variable=caf_var, value="True")
caf_false_radio = ttk.Radiobutton(root, text="False", variable=caf_var, value="False")
caf_true_radio.grid(row=6, column=1, padx=5, pady=5)
caf_false_radio.grid(row=6, column=2, padx=5, pady=5)

# Price
price_label = ttk.Label(root, text="Maximum Price:")
price_label.grid(row=7, column=0, padx=5, pady=5)
price_entry = ttk.Entry(root)
price_entry.grid(row=7, column=1, padx=5, pady=5)

# Submit button
submit_button = ttk.Button(root, text="Submit", command=submit)
submit_button.grid(row=8, columnspan=3, padx=5, pady=5)



root.mainloop()


Input Parameters:
Min Number of Ingredients: 1
Max Number of Ingredients: 10
Alcoholic: True
Special Ingredient: gin
Sugary Score: 1
Acidic: False
Caffeine: False
Price: 20.0


,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,glass,instructions,price
6,Bijou,4,"Orange Bitters, Green Chartreuse, Gin, Sweet Vermouth",True,1,False,False,Cocktail glass,Stir in mixing glass with ice and strain\r\n,16.06
40,Cream Soda,2,"Spiced rum, Ginger ale",True,1,False,False,Highball glass,Pour 1oz of Spiced Rum into a highball glass with ice. Fill with Ginger Ale.,10.00
47,Singapore Sling,6,"Cherry brandy, Grenadine, Gin, Sweet and sour, Carbonated water, Cherry",True,1,False,False,Hurricane glass,Pour all ingredients into cocktail shaker filled with ice cubes. Shake well. Strain into highball glass. Garnish with pineapple and cocktail cherry.,14.49
53,Vesper,3,"Gin, Vodka, Lillet Blanc",True,1,False,False,Cocktail glass,"Shake over ice until well chilled, then strain into a deep goblet and garnish with a thin slice of lemon peel.",11.79
61,Artillery,3,"Sweet Vermouth, Gin, Bitters",True,1,False,False,Cocktail glass,"Stir all ingredients with ice, strain into a cocktail glass, and serve.",11.60
90,Allies Cocktail,3,"Dry Vermouth, Gin, Kummel",True,1,False,False,Cocktail glass,"Stir all ingredients with ice, strain contents into a cocktail glass, and serve.",11.32
106,Adam & Eve,4,"Gin, Cognac, Creme de Cassis, Fresh Lemon Juice",True,1,False,False,Cocktail glass,Shake together all the ingredients and strain into a cold glass.,11.68
170,Waikiki Beachcomber,3,"Triple sec, Gin, Pineapple juice",True,1,False,False,Cocktail glass,"Shake all ingredients with ice, strain into a cocktail glass, and serve.",12.05
235,Queen Elizabeth,3,"Dry Vermouth, Gin, Benedictine",True,1,False,False,Cocktail glass,"Stir all ingredients with ice, strain into a cocktail glass, and serve.",13.95
248,Boomerang,5,"Gin, Dry Vermouth, Bitters, Maraschino liqueur, Maraschino cherry",True,1,False,False,Cocktail glass,"In a mixing glass half-filled with ice cubes, combine the gin, vermouth, bitters, and maraschino liqueur. Stir well. Strain into a cocktail glass and garnish with the cherry.",13.50
